In [6]:
#模拟优矿。
#1.0版本：。
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline #魔法命令，用于在jupyter 环境中正常显示图表
plt.rcParams['font.sans-serif']=['SimHei'] #用来在图中正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来在图中正常显示负号

datename = '221111'
convdataname = '{}{}{}'.format('convdata_',datename,'.csv')
initData=pd.read_csv(convdataname,index_col=0) #读取CSV数据文件。
initData   #83833数据行。

,tradeDate,secID,secShortNameBond,closePriceBond,bondPremRatio,remainSize,convEndtime,traStoptime,Daoqi
0,2022-01-04,110034.XSHG,九州转债,107.730,22.6581,14.984530,2022-01-15,2021-12-31,4
1,2022-01-05,110034.XSHG,九州转债,107.730,24.7289,14.983470,2022-01-15,2021-12-31,5
2,2022-01-06,110034.XSHG,九州转债,107.730,24.4864,14.982680,2022-01-15,2021-12-31,6
3,2022-01-07,110034.XSHG,九州转债,107.730,23.8443,14.981471,2022-01-15,2021-12-31,7
4,2022-01-10,110034.XSHG,九州转债,107.730,25.7085,14.981091,2022-01-15,2021-12-31,10
...,...,...,...,...,...,...,...,...,...
83828,2022-11-02,127073.XSHE,天赐转债,124.600,37.2511,34.105000,2027-09-22,2031-01-01,-2982
83829,2022-11-03,127073.XSHE,天赐转债,125.583,37.9604,34.105000,2027-09-22,2031-01-01,-2981
83830,2022-11-04,127073.XSHE,天赐转债,129.700,32.9963,34.105000,2027-09-22,2031-01-01,-2980
83831,2022-11-07,127073.XSHE,天赐转债,130.300,30.1134,34.105000,2027-09-22,2031-01-01,-2977


In [7]:
TDate=initData['tradeDate'].drop_duplicates()
TDate

0      2022-01-04
1      2022-01-05
2      2022-01-06
3      2022-01-07
4      2022-01-10
          ...    
209    2022-11-02
210    2022-11-03
211    2022-11-04
212    2022-11-07
213    2022-11-08
Name: tradeDate, Length: 204, dtype: object

In [8]:
for today_date in TDate[:3]:
    print(today_date)

2022-01-04
2022-01-05
2022-01-06


In [9]:
Start_Cash= 1000000
MyCash = 1000000   #现金。统计量，后续不断更新～～～～～～～～
HighValue = 0 #最高市值。统计量，后续不断更新～～～～～～～～
Withdraw = 0       #最大回撤。统计量，后续不断更新～～～～～～～～
TValue = 0         #总持仓市值。统计量，后续不断更新～～～～～～～～
HoldNum = 10       #持债支数。
MyPosition = {}    #持仓
Cost = {}          #成本
Value = {}         #市值
Profit = {}        #盈利

In [15]:
for today_date in TDate:
    data = initData.query('tradeDate=="' + today_date +'" and Daoqi < 0 ')
    data.set_index('secShortNameBond',inplace=True)
    data['DoubleLow'] = data['closePriceBond'] + data['bondPremRatio']      #每天重新计算双低排名
    data = data.sort_values(by="closePriceBond" , ascending=True)          #按价格排序
    Tvalue = 0
    for stock in list(MyPosition.keys()): #首先查看持仓盈利情况，先卖后买。通过list遍历，后续删除字典，不会报错！
        CurPrice = data.query('secShortNameBond==@stock').closePriceBond[0] #获取持仓价格
        Value[stock] = MyPosition[stock] * CurPrice * 10                    #更新单只市值
        Tvalue += Value[stock]                                              #更新总持仓市值
        Profit[stock] = round(Value[stock]/Cost[stock]-1,2)                 #更新单只市值
        if Profit[stock] > 0.2:                                             #若盈利20%，则平仓。
            MyCash += Value[stock] * (1-0.0001)                             #卖出处理：现金增加，交易摩擦成本按万分之一。统计量更新～～～
            Tvalue -= Value[stock]                                          #卖出处理：总持仓市值减少。                  统计量更新～～～
            #MyPosition[stock] = 0                                     #卖出处理：持仓数量、成本、市值同步
            del MyPosition[stock]
            print('{} 卖出 {},价格：{},金额：{},持仓市值{} ,现金{} ,'.format
                  (today_date,stock,CurPrice,Value[stock],round(Tvalue,0),round(MyCash,0)) #卖出处理：交易信息输出-------------


    if len(MyPosition) < HoldNum:    #查看持仓数量情况，不足则买。
        targetNum = HoldNum - len(MyPosition)
        BuyMoney = MyCash / targetNum
        for i in range(HoldNum):
            if len(MyPosition) == HoldNum :break
            if data.index[i] not in MyPosition.keys():
                name = data.index[i]                                         #待建仓转债名称
                price = round(data.loc[name]['closePriceBond'],3)            #待建仓转债价格
                cb_ration = round(data.loc[data.index[i]]['bondPremRatio'],2)#待建仓转债溢价率
                MyPosition[name] = int(BuyMoney / price / 10)        #买入处理：持仓数量、成本、市值同步增加。 
                Cost[name] = MyPosition[name] * price * 10           #买入处理：持仓数量、成本、市值同步增加。
                Value[name] = MyPosition[name] * price * 10          #买入处理：持仓数量、成本、市值同步增加。                
                MyCash -= Cost[name]* (1+0.0001)                     #买入处理：现金减少。                   统计量更新～～～
                Tvalue += Value[name]                                #买入处理：总持仓市值增加。             统计量更新～～～           
                message ='{} 买入 {}, 价格 {}, 成本 {},持仓市值{} ,现金{}'.format(today_date, name, price, Cost[name],round(Tvalue,0),round(MyCash,0))
                print(message)                                       #买入处理：交易信息输出----------------------
   #仓位符合要求，则持仓统计：总资产，总持仓市值，总现金，总收益率，最高市值，最大回撤……
    T_asset = MyCash + Tvalue                         #更新总资产                                        统计量更新～～～
    ratio = (T_asset-Start_Cash)/Start_Cash*100       #更新总收益率
    if T_asset > HighValue:HighValue = T_asset        #更新最高市值
    if (HighValue - T_asset) / HighValue > Withdraw: #更新最大回撤 
        Withdraw = (HighValue - T_asset) / HighValue 
    message = '{}:最高市值{} , 总市值{} , 持仓市值{} ,现金{} , 收益率{}% , 最大回撤{}%'.format(today_date,round(HighValue,0),round(T_asset,0),round(Tvalue,0),round(MyCash,0),round(ratio,1),round(Withdraw*100,1))
    print(message)
    print('='*20)
    print('\n')

SyntaxError: invalid syntax (<ipython-input-15-a9e1b00d3c53>, line 21)